In [1]:
import pyBigWig
import numpy as np
"""package github: https://github.com/deeptools/pyBigWig"""

'package github: https://github.com/deeptools/pyBigWig'

In [3]:
def TSS_counter(bw_fwd,bw_rev,gffFile,outFile):

    TSS_total = [0]*200
    TSSfile = open(outFile,"w+")
    gff = open(gffFile)

    for line in gff:
        cols = line.strip().split()
        chrom, cds, start, stop, strand = cols[0], cols[2], int(cols[3]), int(cols[4]), cols[6]
        gene_id = cols[8].split(";")[1].split("Parent=")[1]
        if cds == "CDS":
            if strand == "+":
                if start-200 > 0:
                    UTR_start = start - 200
                else:
                    continue
                TSS_list = [str(value) for value in bw_fwd.values(chrom,UTR_start,start+200)]
                
            else:
                if stop + 200 < bw_rev.chroms()[chrom]:
                    UTR_stop = stop + 200
                else:
                    continue
                TSS_list = [str(abs(value)) for value in bw_rev.values(chrom,stop-200,UTR_stop)][::-1]
            
            TSSfile.write("\t".join([gene_id]+TSS_list))
            TSSfile.write("\n")
         
    bw_fwd.close()
    bw_rev.close()
    gff.close()

In [4]:
gff = "/mnt/brarlab/andrea_higdon/20210111_truncation_peakcaller/TLseq_fulltimecourse/genome_files/SK1_NCSL00000000_SGD.gene.MvO.curated.mRNA.20190128.gff"

bw_0_fwd = pyBigWig.open("p0.MH5p0.TPM.forward.TSS.bw")
bw_0_rev = pyBigWig.open("p0.MH5p0.TPM.reverse.TSS.bw")

bw_2_fwd = pyBigWig.open("p2.MH5p2.TPM.forward.TSS.bw")
bw_2_rev = pyBigWig.open("p2.MH5p2.TPM.reverse.TSS.bw")

bw_3_fwd = pyBigWig.open("p3.MH5p3.TPM.forward.TSS.bw")
bw_3_rev = pyBigWig.open("p3.MH5p3.TPM.reverse.TSS.bw")

bw_4_fwd = pyBigWig.open("p4.MH5p4.TPM.forward.TSS.bw")
bw_4_rev = pyBigWig.open("p4.MH5p4.TPM.reverse.TSS.bw")

bw_5_fwd = pyBigWig.open("p5.MH5p5.TPM.forward.TSS.bw")
bw_5_rev = pyBigWig.open("p5.MH5p5.TPM.reverse.TSS.bw")

bw_6_fwd = pyBigWig.open("p6.MH5p6.TPM.forward.TSS.bw")
bw_6_rev = pyBigWig.open("p6.MH5p6.TPM.reverse.TSS.bw")

bw_7_fwd = pyBigWig.open("p7.MH5p7.TPM.forward.TSS.bw")
bw_7_rev = pyBigWig.open("p7.MH5p7.TPM.reverse.TSS.bw")

bw_8_fwd = pyBigWig.open("p8.MH5p8.TPM.forward.TSS.bw")
bw_8_rev = pyBigWig.open("p8.MH5p8.TPM.reverse.TSS.bw")

bw_9_fwd = pyBigWig.open("p9.MH5p9.TPM.forward.TSS.bw")
bw_9_rev = pyBigWig.open("p9.MH5p9.TPM.reverse.TSS.bw")


In [5]:
TSS_counter(bw_0_fwd,bw_0_rev,gff,"TSScounts_annotated_0h.txt")
TSS_counter(bw_2_fwd,bw_2_rev,gff,"TSScounts_annotated_2h.txt")
TSS_counter(bw_3_fwd,bw_3_rev,gff,"TSScounts_annotated_3h.txt")
TSS_counter(bw_4_fwd,bw_4_rev,gff,"TSScounts_annotated_4h.txt")
TSS_counter(bw_5_fwd,bw_5_rev,gff,"TSScounts_annotated_5h.txt")
TSS_counter(bw_6_fwd,bw_6_rev,gff,"TSScounts_annotated_6h.txt")
TSS_counter(bw_7_fwd,bw_7_rev,gff,"TSScounts_annotated_7h.txt")
TSS_counter(bw_8_fwd,bw_8_rev,gff,"TSScounts_annotated_8h.txt")
TSS_counter(bw_9_fwd,bw_9_rev,gff,"TSScounts_annotated_9h.txt")


In [23]:
def TSS_counter_bed(bw_fwd,bw_rev,bedFile,outFile):

    TSS_total = [0]*200
    TSSfile = open(outFile,"w+")
    bed = open(bedFile)

    for line in bed:
        cols = line.strip().split("\t")
        if cols[0] != "chrom":
            chrom, start, stop, strand = cols[0], int(cols[1]), int(cols[2]), cols[4]
            gene_id = cols[3]
            dist_from_annotated = int(cols[6])
            
            if strand == "+":
                if start-200 > 0:
                    UTR_start = start - 200
                else:
                    continue
                TSS_list = [str(value) for value in bw_fwd.values(chrom,UTR_start,start+200)]
                
                masked_list = []
                for index in range(len(TSS_list)):
                    if index < 200-dist_from_annotated*3:
                        masked_list.append("0")
                    else:
                        masked_list.append(TSS_list[index])
                
            else:
                if stop + 200 < bw_rev.chroms()[chrom]:
                    UTR_stop = stop + 200
                else:
                    continue
                TSS_list = [str(abs(value)) for value in bw_rev.values(chrom,stop-200,UTR_stop)][::-1]
                
                masked_list = []
                for index in range(len(TSS_list)):
                    if index < 200-dist_from_annotated*3:
                        masked_list.append("0")
                    else:
                        masked_list.append(TSS_list[index])
            
            TSSfile.write("\t".join([gene_id]+masked_list))
            TSSfile.write("\n")
         
    bw_fwd.close()
    bw_rev.close()
    bed.close()

In [25]:
bed = ("/mnt/brarlab/andrea_higdon/20210201_LTMandCHX_relignedtoKeeneyLabGenome/20210305_empiricaldist_peakcaller_analysis/20210329_empiricaldist_peakcaller_truncations_p0015_dist5_peakmax11_nomain.csv")

TSS_counter_bed(bw_0_fwd,bw_0_rev,bed,"TSScounts_empiricaldist_truncations_0h.txt")
TSS_counter_bed(bw_2_fwd,bw_2_rev,bed,"TSScounts_empiricaldist_truncations_2h.txt")
TSS_counter_bed(bw_3_fwd,bw_3_rev,bed,"TSScounts_empiricaldist_truncations_3h.txt")
TSS_counter_bed(bw_4_fwd,bw_4_rev,bed,"TSScounts_empiricaldist_truncations_4h.txt")
TSS_counter_bed(bw_5_fwd,bw_5_rev,bed,"TSScounts_empiricaldist_truncations_5h.txt")
TSS_counter_bed(bw_6_fwd,bw_6_rev,bed,"TSScounts_empiricaldist_truncations_6h.txt")
TSS_counter_bed(bw_7_fwd,bw_7_rev,bed,"TSScounts_empiricaldist_truncations_7h.txt")
TSS_counter_bed(bw_8_fwd,bw_8_rev,bed,"TSScounts_empiricaldist_truncations_8h.txt")
TSS_counter_bed(bw_9_fwd,bw_9_rev,bed,"TSScounts_empiricaldist_truncations_9h.txt")